In [1]:
import keras

Using TensorFlow backend.


In [2]:
from keras.applications.xception import Xception
from keras.layers import Input, GlobalAveragePooling2D, Dense
from keras.models import Model
from keras.optimizers import SGD
import numpy as np
import pickle as pkl
import datetime

# Load Cifar10 dataset

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

In [4]:
print('The training set has {0} samples with dimentions {1}.'.format(x_train.shape[0], x_train.shape[1:]))
print('The test set has {0} samples with dimentions {1}.'.format(x_test.shape[0], x_test.shape[1:]))

The training set has 50000 samples with dimentions (32, 32, 3).
The test set has 10000 samples with dimentions (32, 32, 3).


In [5]:
NUM_CLASSES = 10
IMG_ROWS, IMG_COLS = 32, 32
BATCH_SIZE = 512
EPOCHS = 20
VAL_SPLIT = .2
LEARNING_RATE = 1e-2
SGD_MOMENTUM = .9

# Preprocess data

In [6]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [7]:
x_train -= np.mean(x_train)
x_test -= np.mean(x_test)

In [8]:
x_train /= np.std(x_train)
x_test /= np.std(x_test)

In [9]:
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

# Build model

In [10]:
input_tensor = Input(shape=(IMG_ROWS, IMG_COLS, 3))

In [11]:
base_model = Xception(include_top=False, input_tensor=input_tensor)

In [13]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

In [14]:
xception = Model(inputs=base_model.input, outputs=predictions)

In [15]:
xception.compile(optimizer=SGD(lr=LEARNING_RATE, momentum=SGD_MOMENTUM), loss='categorical_crossentropy', metrics=['accuracy'])

# Train model

In [16]:
now = datetime.datetime.now()
tensorflow_log_dir = "Graph/" + now.strftime("%Y%m%d-%H%M%S") + "/"

In [17]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorflow_log_dir, histogram_freq=0,  
          write_graph=False, write_images=False)

In [18]:
xception.fit(x_train, y_train, validation_split=VAL_SPLIT, batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=[tensorboard_callback], verbose=1)

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 55s 1ms/step - loss: 1.7581 - acc: 0.3867 - val_loss: 1.4074 - val_acc: 0.5559
Epoch 2/10
40000/40000 [==============================] - 45s 1ms/step - loss: 0.9821 - acc: 0.6715 - val_loss: 0.9164 - val_acc: 0.6907
Epoch 3/10
40000/40000 [==============================] - 45s 1ms/step - loss: 0.5171 - acc: 0.8236 - val_loss: 0.8931 - val_acc: 0.7225
Epoch 4/10
40000/40000 [==============================] - 45s 1ms/step - loss: 0.2670 - acc: 0.9095 - val_loss: 1.0582 - val_acc: 0.7261
Epoch 5/10
40000/40000 [==============================] - 45s 1ms/step - loss: 0.1286 - acc: 0.9598 - val_loss: 1.2427 - val_acc: 0.7288
Epoch 6/10
40000/40000 [==============================] - 45s 1ms/step - loss: 0.0785 - acc: 0.9749 - val_loss: 1.4798 - val_acc: 0.7186
Epoch 7/10
40000/40000 [==============================] - 45s 1ms/step - loss: 0.0575 - acc: 0.9825 - val_loss: 1.5072 - val_acc

In [19]:
xception.evaluate(x_test, y_test, verbose=1)

10000/10000 [==============================] - 9s 919us/step


[1.8519049719810485, 0.7278]